In [123]:
import sys
sys.path.append("..")

from Conversation.serializers import *
from Conversation.converse.core import Message
from Conversation.converse.prompts import *
from Conversation.converse.tools import *

from turbochat.gptprompts import *

import re
import numpy as np
import pandas as pd
from openai import OpenAI
import cerberus

from Apollo.settings import GPT_KEY

In [83]:

class Message:

    def __init__(self, prompt, context={}, system_instructions=None, history=None, tools: Tools=None) -> None:
        
        self.context = context
        self.prompt = str(prompt)
        self.prompt = self.prompt.format(**context)

        self.system_instructions = system_instructions
        self.history = history

        if tools and not isinstance(tools, Tools):
            raise TypeError("tools should be of instance Tools")
        
        self.tools =  tools
        self.response, self.tool_response = self.call_gpt()
        

    def make_message(self):
        
        system = System(self.system_instructions)
        user_prompt = User(self.prompt)

        if self.history:
            history_messages = Messages.from_text(self.history)
            prompts = [system] + history_messages.prompts + [user_prompt]

        else: prompts = [system, user_prompt]

        return Messages(prompts)
    

    def call_gpt(self):

        message = self.make_message()

        client = OpenAI(api_key=GPT_KEY)
     
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=message.get_entries()
        )


        tool_response = None
        if self.tools:
            tool_response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=message.get_entries(),
                tools = self.tools.get_tools(),
                tool_choice="required" # auto, required, disabled
            )

        return response, tool_response


    def get_history(self, apetite=30):
        # will keep combination of
        messages = self.make_message()
        user = User(self.context.get("message", ""))
        prompts = messages.prompts[1:-1]
        prompts = prompts[apetite*-2:]
        prompts.append(user)
        return Messages(prompts).to_text()


    def get_results(self):

        reply = self.response.choices[0].message.content
        tool_calls = {}
        
        if self.tool_response:
            tool_calls = {}
            tool_calls = self.tools.get_results(self.tool_response, tool_calls)

        return reply, tool_calls


class Prompt:

    def __init__(self, prompt, context, default="") -> None:
        self.prompt = str(prompt)
        self.maps = self.consume(prompt, context, default="")


    def consume(self, prompt, context={}, default=""):

        required_context = re.findall(r'\{([^}]*)\}', USER_PROMPT_DEFAULT)
        for key in required_context:
            if key not in context: context[key] = default

        prompt = prompt.format(**context)

        splits = [x.strip('\n') for x in prompt.split("#")]
        splits = [x.split('\n') for x in splits if x]
        splits = {x[0].strip().replace(":", ""): "\n".join(x[1:]).strip() for x in splits if len(x) >= 2}
        
        return splits


    def get_format(self, keys: list, key_value_format=lambda k, v: f"{k}:\n{v}", join_with='\n\n'):
        
        if not all(key in self.maps for key in keys):
            raise ValueError("not all keys present in the prompt")
        
        outs = [key_value_format(key, self.maps[key]) for key in keys]
        return join_with.join(outs)




In [35]:
def collected_health_information_entries(entries):
    df = pd.DataFrame(entries)
    df.columns = ["i_parameter_label", "parameter_type", "parameter_value"]
    return df

CONVERSE_TOOLS = Tools([
    {
        "name": "collected_health_information_entries",
        "function": collected_health_information_entries,
        "definition": EXTRACT_USER_RELATED_INFO
    }
])

In [36]:
questions_answered_tool_definition = {
    "type": "function",
    "function": {
        "name": "questions_answered",
        "description": """
            This function will be called whenever the previous response from assistant has questions.
            looking at assistant previous response and user message determine's which questions were answered by the user.
            this function will list down all the questions and answers to them provided by user.
            If user has provided no answers set answer as "NAN"
            If no questions where asked return empty
        """,
        "parameters": {
            "type": "object",
            "properties": {
                "questions": {
                    "type": "array",
                    "description": "list of questions asked by assistant and their answers provided by user in user message",
                    "items": {
                        "type": "object",
                        "properties": {
                            
                            "question": {
                                "type": "string",
                                "description": """
                                    question asked by assistant
                                """
                            },


                            "subject": {
                                "type": "string",
                                "description": """
                                    one word subject of the question.
                                """
                            },
                            
                            "answer": {
                                "type": "string",
                                "description": """
                                    answer provided in the user message.
                                    If answer is not provided it should be set to "NAN"
                                """
                            },
                        
                        },
                        "required": ["questions", "subject", "answer"],
                    }
                }
            },
        },
        "required": ["questions"],
    },
}


user_other_mentions_tool_definition = {
    "type": "function",
    "function": {
        "name": "user_other_mentions",
        "description": """
            This function will be called whenever user mentions something in response to assistant.
            From assistant previous response and user message identify what user is trying to mention which is not asked by the assistant.
            If no questions where asked return empty
        """,
        "parameters": {
            "type": "object",
            "properties": {
                "mentions": {
                    "type": "array",
                    "description": "list of other mentions by the user",
                    "items": {
                        "type": "object",
                        "properties": {
                            
                            "context": {
                                "type": "string",
                                "description": """
                                    very short context to what user is trying to mention with the reasons if any mentioned.
                                """
                            },


                            "subject": {
                                "type": "string",
                                "description": """
                                    one word subject for the user mentioned context.
                                """
                            },
                            
                            "key_highlight": {
                                "type": "string",
                                "description": """
                                    1 word key that was highligted in the user's context that will describe the subject
                                """
                            },
                        
                        },
                        "required": ["context", "subject", "key_highlight"],
                    }
                }
            },
        },
        "required": ["mentions"],
    },
}


In [81]:
SYSTEM_MESSAGE = """
ALWAYS KEEP CONVERSATION SHORT AND TO THE POINT.
Collect as much information about the user as possible by asking QUESTIONS and chaining conversation.
Collect information regarding their habits, mental, physical health, medical conditions, reports, tests, lifestyle choices preferences etc. Everything which is an indicator of a user's health.
Always ask for medical test reports related to user's health issues. If user does not have one suggest them to take the required tests. Also list them what reports/tests are required for the same.
Give more importance information provided through medical tests/reports by the user.
Whatever user enquires about ask follow up QUESTIONS around the topic to engage the user in multiple conversation chain.
If conversation goes off topic, acknowledge the user's responses and get the conversation back to health and lifestyle.
Suggest small goals to user in order to resolve their problem.
User's medical history will be provided in the user's response, ALWAYS consider the user's past history while collecting user's information.
ALWAYS END YOUR CONVERSATION WITH A QUESTION. NEVER STOP A CONVERSATION. ONCE THE CONVERSATION CHAIN IS OVER QUICKLY CHANGE TO RELATED TOPICS BY ASKING MORE QUESTIONS. KEEP THE CONVERSATION SHORT. DONT ASK EVERYTHING IN THE SAME RESPONSE.
If you think user is mistaken, give response with reference to user's previous prompt.
"""

USER_PROMPT_DEFAULT = """

# conversation state:
{state}


# conversation topic:
{topic}


# user history:
{history}


# user goals:
{goals}


# user events:
{events}


# Available provider:
{providers}


# Available doctors:
{doctors}


# assistant previous response:
{assistant_previous_response}


# user reply:
{user_reply}


"""




In [90]:

def to_df(questions):
    return pd.DataFrame(questions)


CONVERSE_TOOLS = Tools([
    # {
    #     "name": "questions_answered",
    #     "function": lambda questions: pd.DataFrame(questions),
    #     "definition": questions_answered_tool_definition
    # },
    {
        "name": "user_other_mentions",
        "function": lambda mentions: pd.DataFrame(mentions),
        "definition": user_other_mentions_tool_definition
    },
])

question_was_asked_and_answered_context = {
    "state": "engaging",
    "assistant_previous_response": "It's important to listen to your body and take care of yourself, especially if you feel like you've been overworking recently. Have you noticed any specific symptoms or signs of overwork, such as fatigue, muscle pain, or trouble sleeping? How have you been managing your stress levels and self-care practices lately? It might be helpful to prioritize relaxation and self-care activities to help you recharge. Do you have any upcoming appointments with a healthcare provider, or have you discussed this with a doctor or therapist before?",
    "user_reply": "Yes I do have problem sleeping and red eyes in morning after I wake up."
}

prompts = Prompt(USER_PROMPT_DEFAULT, question_was_asked_and_answered_context)
print(prompts.get_format(["assistant previous response", "user reply"]))

# message = Message(USER_PROMPT_DEFAULT, question_was_asked_and_answered_context, SYSTEM_MESSAGE, history=None, tools=CONVERSE_TOOLS)
# reply, tool_calls = message.get_results()

assistant previous response:
It's important to listen to your body and take care of yourself, especially if you feel like you've been overworking recently. Have you noticed any specific symptoms or signs of overwork, such as fatigue, muscle pain, or trouble sleeping? How have you been managing your stress levels and self-care practices lately? It might be helpful to prioritize relaxation and self-care activities to help you recharge. Do you have any upcoming appointments with a healthcare provider, or have you discussed this with a doctor or therapist before?

user reply:
Yes I do have problem sleeping and red eyes in morning after I wake up.


In [104]:
req = message.tool_response

req.choices[0].message.content

In [107]:

from typing import Any

from turbochat.gptprompts import Messages


class GPT:


    def __init__(self, api_key, model="gpt-3.5-turbo") -> None:
        self.client = OpenAI(api_key=GPT_KEY)
        self.model = model

    
    def call(self, **kwargs):
        return self.client.chat.completions.create(model=self.model, **kwargs)


    @staticmethod
    def get_response_entry(response, return_first=True):
        
        if len(response.choices):
            
            if return_first: return response.choices[0].message.to_dict()
            else:
                return [msg.to_dict() for msg in response.choices]

        return None
    

    @staticmethod
    def get_reply(response, return_first=True):
        if len(response.choices):
            
            if return_first: return response.choices[0].message.content
            else:
                return [msg.content for msg in response.choices]


class GPTCallable:

    def __init__(self, gpt: GPT, messages: Messages):

        if not isinstance(gpt, GPT): raise TypeError("expected type for gpt: GPT")
        
        if not isinstance(messages, Messages): raise TypeError("expected type for messages: Messages")

        self.gpt = gpt
        self.messages = messages


    def _call(self): raise NotImplementedError

    
    def call(self):
        response = self._call(self)
        response_entry = GPT.get_response_entry(response)
        return response, response_entry


class ToolCall(GPTCallable):


    def __init__(self, gpt: GPT, messages: Messages, tool: Tools, required=False):
        super().__init__(gpt, messages)

        if not isinstance(tool, Tools): raise TypeError("expected type for tool: Tools")
        
        self.tool = tool
        self.required = required

    
    def _call(self):
        response = self.gpt.call(messages=self.messages.get_entries(), tools=self.tool.get_tools(), tool_choice=self.required)
        return response
    

    def call(self):
        response, response_entry = super().call()
        results = self.get_tool_results(response)
        return response, response_entry, results


    def get_tool_results(self, response):
        result = {}
        result = self.tool.get_results(response, result)
        return result


class Msg(GPTCallable):

    def __init__(self, gpt: GPT, messages: Messages):
        super().__init__(gpt, messages)

        self.gpt = gpt
        self.messages = messages

    def _call(self):
        response = self.gpt.call(messages=self.messages.get_entries())
        return response

    def call(self):
        response, response_entry = super().call()
        result = GPT.get_reply(response)
        return response, response_entry, result




In [122]:
# gpt = GPT(GPT_KEY, model="gpt-4o")


USER_CONTENT_SCHEMA = {
    "role": {"type": "string", "required": True, "empty": False, "allowed": ["user"]},
    "content": {
        "type": "list", "required": True, "empty": False, "nullable": True, "schema": {

            "anyof": [

                {
                    "type": "dict", "empty": False, "schema": {

                        "type": { "type": "string", "required": True, "empty": False, "allowed": ["text"] },
                        "content": { "type": "string", "required": True, "empty": True }

                    }
                },


                {
                    "type": "dict", "empty": False, "schema": {

                        "type": { "type": "string", "required": True, "empty": False, "allowed": ["image_url"] },
                        "image_url": { "type": "string", "required": True, "empty": False, "check_with": "image_url" }

                    }
                },

            ]
        }
    }
}


gpt = GPT(GPT_KEY)
# response = gpt.call(
#     messages=[
#     {
#       "role": "user",
#       "content": [
#         {
#           "type": "text",
#           "text": "What are in these images? Is there any difference between them?",
#         },
#         {
#           "type": "text",
#           "text": "What are in these images? Is there any difference between them?",
#         }
#       ],
#     }
#   ],
# )

In [121]:
response.choices[0].message.to_dict()

{'content': "I'm sorry, but as an AI text-based model, I am unable to view images. Could you please provide a description or context for the images so that I can assist you better?",
 'role': 'assistant'}

In [72]:
print(reply)
print('\n')

for key in tool_calls:
    result = tool_calls[key]
    # print(result)
    print(result[0].to_markdown(index=False))
    print('\n')



I'm sorry to hear about your sleeping issues and red eyes in the morning. It's important to address these symptoms to improve your overall health. Have you ever had a sleep study done to check for any sleep disorders such as sleep apnea? This test can provide valuable insights into your sleeping patterns and help identify any underlying issues that may be affecting your sleep quality. Would you be willing to consider undergoing a sleep study to investigate your sleep problems further?


| context             | subject   | key_highlight   |
|:--------------------|:----------|:----------------|
| problem sleeping    | sleeping  | sleeping        |
| red eyes in morning | eyes      | eyes            |




In [ ]:
question_was_asked_and_answered_context = {
    "state": "engaging",
    "topic": "",
    "history": "",
    "goals": "",
    "events": "",
    "providers": "",
    "doctors": "",
    "assistant_previous_response": "It's important to listen to your body and take care of yourself, especially if you feel like you've been overworking recently. Have you noticed any specific symptoms or signs of overwork, such as fatigue, muscle pain, or trouble sleeping? How have you been managing your stress levels and self-care practices lately? It might be helpful to prioritize relaxation and self-care activities to help you recharge. Do you have any upcoming appointments with a healthcare provider, or have you discussed this with a doctor or therapist before?",
    "user_reply": "Yes I do have problem sleeping and red eyes in morning after I wake up."
}